In [ ]:
import os
path = "D:\\dogs-vs-cats"
training_data_dir = os.path.join(path,"train") # 10 000 * 2
validation_data_dir = os.path.join(path,"validation") # 2 500 * 2
test_data_dir = os.path.join(path,"test") # 12 500

In [ ]:
import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
training_data_generator = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True)
validation_data_generator = ImageDataGenerator(rescale=1./255)
test_data_generator = ImageDataGenerator(rescale=1./255)

In [ ]:
training_generator = training_data_generator.flow_from_directory(
    training_data_dir,
    target_size=(200, 200),
    batch_size=32,
    class_mode="binary")
validation_generator = validation_data_generator.flow_from_directory(
    validation_data_dir,
    target_size=(200, 200),
    batch_size=32,
    class_mode="binary")
test_generator = test_data_generator.flow_from_directory(
    test_data_dir,
    target_size=(200, 200),
    batch_size=1,
    class_mode="binary", 
    shuffle=False)

In [ ]:
from keras import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(16))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation("sigmoid"))

model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=["accuracy"])
print(model.summary())

In [ ]:
model.fit_generator(
    training_generator,
    steps_per_epoch=len(training_generator.filenames) // 32,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=len(validation_generator.filenames) // 32)

In [ ]:
from keras.models import load_model
from keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
import os

def load_image(img_path, show=True):

    img = image.load_img(img_path, target_size=(200, 200))
    img_tensor = image.img_to_array(img)                   
    img_tensor = np.expand_dims(img_tensor, axis=0)         
    img_tensor /= 255.                                      

    if show:
        plt.imshow(img_tensor[0])                           
        plt.axis('off')
        plt.show()

    return img_tensor

img_path = os.path.join(test_data_dir,"5.jpg")
new_image = load_image(img_path)
pred = model.predict(new_image)
ans = print(pred)
if pred > 0.5:
    print('dog')
else:
    print('cat')